In [11]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import Markdown, display, HTML
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
from datetime import date

load_dotenv()

llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Using Sample Case - 1

In [12]:
system_prompt = """You are a clinical scribe. Given this doctor-patient dialogue and metadata, generate a SOAP-formatted visit summary with sections:  
  - Chief Complaint  
  - History of Present Illness  
  - Review of Systems (if mentioned)  
  - Physical Exam Findings (if mentioned)  
  - Assessment & Plan (including diagnoses, orders, prescriptions, follow-up)  
  - Patient Instructions & Education
"""

In [13]:
user_input = """Dr. Harris: Good morning, Mr. Garcia. How have you been feeling since our last visit?
José Garcia: Morning, doc. I've been feeling okay overall, but lately my right foot has started burning and tingling a lot, especially when I get up after sitting.
Dr. Harris: I understand. Let me quickly confirm some things before we dive deeper. You're 67 years old, retired from construction, and living alone, correct?
José Garcia: Yes, that's right.
Dr. Harris: And you still have type 2 diabetes, high blood pressure, high cholesterol, and stage 3 chronic kidney disease. Am I missing anything?
José Garcia: No, you got it all. And the diabetes has been there for about 15 years now.
Dr. Harris: Got it. Any changes recently in your diet or your daily routine?
José Garcia: Honestly, transportation is still tough, and sometimes I skip meals because it's hard getting to the grocery store. Money can be tight, you know.
Dr. Harris: I completely understand; thanks for letting me know. This matters a lot for your overall health. How about medications—still on Metformin, Lisinopril, Atorvastatin, Gabapentin, and Aspirin?
José Garcia: Yeah, exactly those. Haven't missed doses, but the Gabapentin doesn't seem to help with the foot pain anymore.
Dr. Harris: Alright. Nurse Kelly will take your vitals quickly, then we'll check out your foot.
(Nurse Kelly takes vital signs.)
Nurse Kelly: Your blood pressure is 148 over 88, heart rate 80, oxygen is good at 96%, and your weight today is 82 kilos.
Dr. Harris: Your blood pressure is a bit high today, José. We'll keep an eye on that. How's your blood sugar management lately?
José Garcia: It's not great, honestly. The last A1C was about 8.2%. I admit, I've had some trouble controlling it.
Dr. Harris: Thanks for being honest about that. Let's check your feet. (Examines feet) Hmm, your pulses are a bit weak, and your sensation is reduced in your toes. There's no open wound, but there are some calluses.
José Garcia: Is that what's causing the burning feeling?
Dr. Harris: Yes, it's likely due to diabetic neuropathy. This happens when high sugars damage nerves over time. I'll prescribe Duloxetine—it can help manage nerve pain more effectively.
José Garcia: Sounds good, doc. Anything else I should do?
Dr. Harris: Yes, I'd like you to see a podiatrist. They'll manage those calluses and teach you foot care. Also, I want to involve our care manager, Mrs. Lee. She can help you access better transportation and some meal support programs.
(Mrs. Lee, care manager, joins briefly.)
Mrs. Lee: Hi José! I'll call you each week to see how your blood sugars are doing and help arrange transportation and meals. Would that be helpful?
José Garcia: Yes, thank you! I really appreciate it.
Dr. Harris: Excellent. Let's also give you the pneumonia vaccine today. We'll re-check labs next month. Does that sound good?
José Garcia: Sure, thanks again, doc.
"""

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)

response = llm.invoke(prompt.format(input=user_input))

In [15]:
display(Markdown(response.content))

**SOAP Visit Summary**

**Chief Complaint:**  
Burning and tingling sensation in the right foot, especially after sitting.

**History of Present Illness:**  
José Garcia, a 67-year-old male with a history of type 2 diabetes, hypertension, hyperlipidemia, and stage 3 chronic kidney disease, reports a recent onset of burning and tingling in his right foot. These symptoms are particularly noticeable when he stands up after sitting. He has been on Metformin, Lisinopril, Atorvastatin, Gabapentin, and Aspirin. He reports that Gabapentin is no longer effective for his foot pain. He also mentions difficulties with transportation and occasionally skipping meals due to financial constraints.

**Review of Systems:**  
- General: No recent changes in overall health.
- Neurological: Burning and tingling in the right foot.
- Endocrine: Poor blood sugar control, with a recent A1C of 8.2%.

**Physical Exam Findings:**  
- Vitals: Blood pressure 148/88 mmHg, heart rate 80 bpm, oxygen saturation 96%, weight 82 kg.
- Foot Exam: Weak pulses and reduced sensation in toes, presence of calluses, no open wounds.

**Assessment & Plan:**  
1. **Diabetic Neuropathy:**  
   - Prescribe Duloxetine for better management of nerve pain.
   - Refer to a podiatrist for callus management and foot care education.

2. **Type 2 Diabetes Mellitus:**  
   - Poor glycemic control noted with A1C of 8.2%.
   - Involve care manager to assist with transportation and meal support programs.
   - Weekly follow-up calls from care manager to monitor blood sugar levels.

3. **Hypertension:**  
   - Blood pressure slightly elevated at 148/88 mmHg. Continue monitoring.

4. **Preventive Care:**  
   - Administer pneumonia vaccine today.

5. **Follow-up:**  
   - Re-check labs in one month.

**Patient Instructions & Education:**  
- Start taking Duloxetine as prescribed for foot pain.
- Attend podiatry appointment for foot care and callus management.
- Engage with care manager for assistance with transportation and meal support.
- Monitor blood sugar levels regularly and report any significant changes.
- Return for lab work in one month and maintain regular follow-up appointments.

## Single Shot (One Example)

In [16]:
example = """
Conversation : Doctor-Patient Dialogue>

Dr. Yazzie: Michael, what's bringing you in today?
Michael Red Cloud: Feeling more tired, legs swollen by the evening.
Dr. Yazzie: Sounds like kidney issues progressing. Let's check your vitals.
(Nurse White takes vitals.)
Nurse White: BP 155/90, weight 95 kg, up 3 kg, noticeable leg swelling.
Dr. Yazzie: Kidney function likely worsening. We'll adjust your diuretic and start pre-dialysis discussions. Telehealth care management will also support you at home.
(Ms. Morningstar from care management briefly joins.)
Ms. Morningstar: I'll coordinate transport, home dialysis education, and food support resources.
Michael Red Cloud: Thank you so much.
 
Structured SOAP Note:

Subjective: Fatigue, leg swelling, history CKD Stage 4, diabetes, hypertension, limited rural resources.
Objective: BP 155/90, edema 2+ lower limbs, increased weight.
Assessment: CKD worsening, uncontrolled HTN, depression, SDOH challenges.
Plan: Furosemide BID, pre-dialysis referral, telehealth counseling, tribal resources for food, follow-up monthly.
Billing: CPT 99215, ICD-10 N18.4, E11.22, I10, F33.1, Z59.6
"""

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt + "\n\nHere is an example:\n" + example),
        ("user", "{input}"),
    ]
)
response = llm.invoke(prompt.format(input=user_input))
display(Markdown(response.content))

**SOAP Note**

**Subjective:**
- Chief Complaint: Burning and tingling sensation in the right foot, especially after sitting.
- History of Present Illness: José Garcia, a 67-year-old male with a history of type 2 diabetes (15 years), hypertension, hyperlipidemia, and stage 3 chronic kidney disease, reports increased burning and tingling in the right foot. He lives alone and faces transportation challenges, leading to occasional skipped meals. He is currently on Metformin, Lisinopril, Atorvastatin, Gabapentin, and Aspirin. Gabapentin is no longer effective for foot pain. Recent A1C was 8.2%.

**Objective:**
- Vitals: BP 148/88 mmHg, HR 80 bpm, SpO2 96%, Weight 82 kg.
- Physical Exam Findings: Weak pulses and reduced sensation in toes, presence of calluses, no open wounds.

**Assessment:**
- Diabetic neuropathy with inadequate pain control.
- Suboptimal diabetes management (A1C 8.2%).
- Hypertension, not optimally controlled.
- Social determinants of health impacting nutrition and healthcare access.

**Plan:**
1. **Medications:**
   - Prescribe Duloxetine for neuropathic pain management.
2. **Referrals:**
   - Podiatry referral for callus management and foot care education.
3. **Care Management:**
   - Involve Mrs. Lee, care manager, to assist with transportation and meal support programs.
4. **Vaccinations:**
   - Administer pneumonia vaccine today.
5. **Follow-up:**
   - Re-check labs in one month.
   - Weekly follow-up calls from care manager to monitor blood sugar and assist with resources.

**Patient Instructions & Education:**
- Educate on diabetic foot care and the importance of regular foot checks.
- Discuss the role of Duloxetine in managing nerve pain.
- Encourage adherence to medication and dietary recommendations.
- Inform about the importance of regular follow-ups and lab checks to monitor diabetes and kidney function.

## Modified System Prompt

In [18]:
system_prompt_detailed = """
You are an AI clinical scribe that generates comprehensive SOAP-formatted visit summaries from doctor-patient dialogues. Your documentation should be thorough, accurate, and professionally structured.

## Required Documentation Structure:

### SUBJECTIVE:
- **Patient Demographics**: Age, gender, ethnicity (when mentioned)
- **Chief Complaint**: Primary reason for visit in patient's own words
- **History of Present Illness**: Detailed narrative of current symptoms, timeline, severity, aggravating/alleviating factors
- **Medical History**: Chronic conditions, duration of diagnoses
- **Social History**: Living situation, occupation, transportation issues, financial constraints, food security
- **Family History**: Relevant hereditary conditions (when mentioned)
- **Current Medications**: Complete list with dosages and frequency
- **Review of Systems**: Any additional symptoms mentioned during conversation

### OBJECTIVE:
- **Vital Signs**: All measurements taken (BP, HR, RR, Temp, SpO₂, Weight, BMI when calculable)
- **Physical Examination**: Detailed findings from any body systems examined
- **Diagnostic Tests**: Results of any tests performed or mentioned

### ASSESSMENT:
- **Primary Diagnoses**: Listed in order of clinical priority
- **Secondary Diagnoses**: Chronic conditions and comorbidities
- **Social Determinants of Health**: Transportation, housing, food insecurity, financial strain
- **Psychosocial Factors**: Mental health, stress, family dynamics

### PLAN:
- **Medications**: New prescriptions, dosage changes, discontinuations
- **Diagnostic Orders**: Labs, imaging, other tests ordered
- **Referrals**: Specialist consultations, allied health services
- **Care Coordination**: Care management involvement, home health services
- **Patient Education**: Instructions provided to patient
- **Follow-up**: Next appointment timing and purpose
- **Preventive Care**: Vaccinations, screenings administered

### BILLING INFORMATION:
- **CPT Code**: Appropriate evaluation and management code
- **ICD-10 Codes**: All relevant diagnosis codes including social determinants

## Key Documentation Principles:

1. **Capture Social Context**: Always document social determinants of health, including transportation barriers, food insecurity, housing issues, and financial constraints
2. **Include Care Team**: Document involvement of nurses, care managers, and other healthcare team members
3. **Quantify When Possible**: Include specific measurements, timeframes, and severity indicators
4. **Maintain Professional Tone**: Use appropriate medical terminology while preserving important patient quotes
5. **Comprehensive Assessment**: Address both medical and psychosocial aspects of care
6. **Billing Accuracy**: Ensure CPT codes reflect visit complexity and ICD-10 codes capture all diagnoses and social factors

## Special Attention Areas:
- Chronic disease management and control status
- Medication adherence and effectiveness
- Care coordination efforts
- Social barriers to health
- Patient education provided
- Preventive care opportunities

Generate documentation that would meet standards for clinical care, billing compliance, and quality reporting while maintaining the narrative flow that captures the complete patient encounter."""

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_detailed),
        ("user", "{input}"),
    ]
)

response = llm.invoke(prompt.format(input=user_input))
display(Markdown(response.content))

### SUBJECTIVE:

- **Patient Demographics**: José Garcia, 67-year-old male, retired from construction, living alone.
- **Chief Complaint**: "My right foot has started burning and tingling a lot, especially when I get up after sitting."
- **History of Present Illness**: Mr. Garcia reports a burning and tingling sensation in his right foot, particularly noticeable upon standing after sitting. The symptoms have been persistent and are not alleviated by his current medication, Gabapentin.
- **Medical History**: 
  - Type 2 diabetes for 15 years
  - Hypertension
  - Hyperlipidemia
  - Stage 3 chronic kidney disease
- **Social History**: Lives alone, retired from construction. Experiences transportation difficulties and financial constraints, leading to occasional skipped meals due to difficulty accessing groceries.
- **Family History**: Not mentioned.
- **Current Medications**: 
  - Metformin
  - Lisinopril
  - Atorvastatin
  - Gabapentin
  - Aspirin
- **Review of Systems**: Reports burning and tingling in the right foot. No open wounds but presence of calluses.

### OBJECTIVE:

- **Vital Signs**: 
  - Blood Pressure: 148/88 mmHg
  - Heart Rate: 80 bpm
  - SpO₂: 96%
  - Weight: 82 kg
- **Physical Examination**: 
  - Foot examination reveals weak pulses and reduced sensation in toes. Presence of calluses, no open wounds.
- **Diagnostic Tests**: Last reported A1C was 8.2%.

### ASSESSMENT:

- **Primary Diagnoses**: 
  - Diabetic neuropathy
- **Secondary Diagnoses**: 
  - Type 2 diabetes mellitus
  - Hypertension
  - Hyperlipidemia
  - Stage 3 chronic kidney disease
- **Social Determinants of Health**: Transportation difficulties, financial constraints affecting food security.
- **Psychosocial Factors**: Stress related to managing chronic conditions and financial limitations.

### PLAN:

- **Medications**: 
  - Prescribe Duloxetine for neuropathic pain management.
- **Diagnostic Orders**: 
  - Re-check labs next month.
- **Referrals**: 
  - Podiatrist for foot care and callus management.
- **Care Coordination**: 
  - Involve care manager, Mrs. Lee, to assist with transportation and meal support programs.
- **Patient Education**: 
  - Educated on diabetic foot care and importance of blood sugar control.
- **Follow-up**: 
  - Weekly calls from care manager to monitor blood sugar and assist with logistics.
- **Preventive Care**: 
  - Administer pneumonia vaccine.

### BILLING INFORMATION:

- **CPT Code**: 99214 (Established patient, moderate complexity)
- **ICD-10 Codes**: 
  - E11.40 (Type 2 diabetes mellitus with diabetic neuropathy)
  - I10 (Hypertension)
  - E78.5 (Hyperlipidemia)
  - N18.3 (Chronic kidney disease, stage 3)
  - Z59.4 (Lack of adequate food and safe drinking water)
  - Z59.82 (Transportation insecurity)

# Adding Structured Output

In [20]:
class VitalSigns(BaseModel):
    blood_pressure_systolic: Optional[int] = Field(
        None, description="Systolic BP in mmHg"
    )
    blood_pressure_diastolic: Optional[int] = Field(
        None, description="Diastolic BP in mmHg"
    )
    heart_rate: Optional[int] = Field(None, description="Heart rate in bpm")
    respiratory_rate: Optional[int] = Field(
        None, description="Respiratory rate per minute"
    )
    temperature: Optional[float] = Field(None, description="Temperature in Fahrenheit")
    oxygen_saturation: Optional[int] = Field(None, description="SpO2 percentage")
    oxygen_saturation_on_room_air: Optional[bool] = Field(
        None, description="Whether O2 sat is on room air"
    )
    weight_kg: Optional[float] = Field(None, description="Weight in kilograms")
    bmi: Optional[float] = Field(None, description="Body Mass Index")
    weight_change: Optional[str] = Field(
        None, description="Weight change from previous visit"
    )


class Medication(BaseModel):
    name: str = Field(..., description="Medication name")
    dosage: str = Field(..., description="Dosage amount")
    frequency: str = Field(..., description="Frequency (e.g., BID, daily, TID)")
    route: Optional[str] = Field(None, description="Route of administration")


class SocialHistory(BaseModel):
    living_situation: Optional[str] = Field(None, description="Who patient lives with")
    occupation: Optional[str] = Field(None, description="Current or former occupation")
    employment_status: Optional[str] = Field(None, description="Working, retired, etc.")
    transportation_barriers: Optional[bool] = Field(
        None, description="Has transportation difficulties"
    )
    food_insecurity: Optional[bool] = Field(
        None, description="Experiences food insecurity"
    )
    financial_strain: Optional[bool] = Field(
        None, description="Has financial difficulties"
    )
    housing_stability: Optional[str] = Field(None, description="Housing situation")


class PhysicalExam(BaseModel):
    general_appearance: Optional[str] = Field(None, description="General appearance")
    cardiovascular: Optional[str] = Field(
        None, description="Cardiovascular exam findings"
    )
    respiratory: Optional[str] = Field(None, description="Respiratory exam findings")
    extremities: Optional[str] = Field(None, description="Extremity exam findings")
    neurological: Optional[str] = Field(None, description="Neurological exam findings")
    skin: Optional[str] = Field(None, description="Skin exam findings")
    musculoskeletal: Optional[str] = Field(None, description="Musculoskeletal findings")
    other_findings: Optional[str] = Field(
        None, description="Other examination findings"
    )


class Diagnosis(BaseModel):
    condition: str = Field(..., description="Diagnosis or condition name")
    icd10_code: Optional[str] = Field(None, description="ICD-10 diagnosis code")
    status: Optional[str] = Field(None, description="Active, stable, worsening, etc.")
    severity: Optional[str] = Field(None, description="Mild, moderate, severe")


class Medication_Plan(BaseModel):
    medication: str = Field(..., description="Medication name")
    action: str = Field(
        ..., description="start, increase, decrease, discontinue, continue"
    )
    dosage: str = Field(..., description="New or current dosage")
    frequency: str = Field(..., description="Dosing frequency")
    indication: Optional[str] = Field(None, description="What condition this treats")


class Order(BaseModel):
    type: str = Field(..., description="lab, imaging, procedure, etc.")
    description: str = Field(..., description="Specific test or procedure ordered")
    indication: Optional[str] = Field(None, description="Clinical reason for order")
    timing: Optional[str] = Field(None, description="When to complete")


class Referral(BaseModel):
    specialty: str = Field(..., description="Type of specialist or service")
    reason: str = Field(..., description="Reason for referral")
    urgency: Optional[str] = Field(None, description="Routine, urgent, stat")


class CareCoordination(BaseModel):
    care_manager_involved: Optional[bool] = Field(
        None, description="Care manager participated"
    )
    care_manager_name: Optional[str] = Field(None, description="Name of care manager")
    services_coordinated: Optional[List[str]] = Field(
        None, description="Services arranged"
    )
    home_health_ordered: Optional[bool] = Field(
        None, description="Home health services ordered"
    )
    social_services_referral: Optional[bool] = Field(
        None, description="Social services involved"
    )


class FollowUp(BaseModel):
    timing: str = Field(
        ..., description="When to follow up (e.g., '2 weeks', '1 month')"
    )
    location: Optional[str] = Field(None, description="Clinic, telehealth, etc.")
    purpose: Optional[str] = Field(None, description="Reason for follow-up visit")


class BillingInfo(BaseModel):
    cpt_code: str = Field(..., description="CPT code for visit")
    visit_complexity: Optional[str] = Field(
        None, description="Low, moderate, high complexity"
    )
    icd10_codes: List[str] = Field(..., description="All ICD-10 codes for billing")


class Subjective(BaseModel):
    patient_name: Optional[str] = Field(None, description="Patient name")
    age: Optional[int] = Field(None, description="Patient age")
    gender: Optional[str] = Field(None, description="Patient gender")
    ethnicity: Optional[str] = Field(None, description="Patient ethnicity")
    chief_complaint: str = Field(..., description="Primary reason for visit")
    history_present_illness: str = Field(..., description="Detailed HPI narrative")
    medical_history: List[str] = Field(
        default_factory=list, description="Past medical history"
    )
    surgical_history: Optional[List[str]] = Field(
        None, description="Past surgical history"
    )
    family_history: Optional[str] = Field(None, description="Relevant family history")
    social_history: Optional[SocialHistory] = Field(
        None, description="Social history details"
    )
    current_medications: List[Medication] = Field(
        default_factory=list, description="Current medications"
    )
    allergies: Optional[List[str]] = Field(None, description="Known allergies")
    review_of_systems: Optional[str] = Field(
        None, description="Additional symptoms mentioned"
    )


class Objective(BaseModel):
    vital_signs: Optional[VitalSigns] = Field(
        None, description="Vital sign measurements"
    )
    physical_exam: Optional[PhysicalExam] = Field(
        None, description="Physical examination findings"
    )
    diagnostic_results: Optional[Dict[str, str]] = Field(
        None, description="Lab or test results mentioned"
    )


class Assessment(BaseModel):
    primary_diagnoses: List[Diagnosis] = Field(
        ..., description="Primary diagnoses in order of priority"
    )
    secondary_diagnoses: Optional[List[Diagnosis]] = Field(
        None, description="Secondary/chronic diagnoses"
    )
    social_determinants: Optional[List[str]] = Field(
        None, description="Social factors affecting health"
    )
    clinical_impression: Optional[str] = Field(
        None, description="Overall clinical assessment"
    )


class Plan(BaseModel):
    medications: Optional[List[Medication_Plan]] = Field(
        None, description="Medication changes"
    )
    orders: Optional[List[Order]] = Field(None, description="Diagnostic orders")
    referrals: Optional[List[Referral]] = Field(
        None, description="Specialist referrals"
    )
    care_coordination: Optional[CareCoordination] = Field(
        None, description="Care team coordination"
    )
    patient_education: Optional[List[str]] = Field(
        None, description="Education provided"
    )
    follow_up: Optional[FollowUp] = Field(None, description="Follow-up plans")
    preventive_care: Optional[List[str]] = Field(
        None, description="Vaccines, screenings provided"
    )


class SOAPNote(BaseModel):
    """
    Complete SOAP note structure for clinical documentation
    """

    visit_date: Optional[date] = Field(None, description="Date of visit")
    subjective: Subjective = Field(..., description="Subjective section")
    objective: Objective = Field(..., description="Objective section")
    assessment: Assessment = Field(..., description="Assessment section")
    plan: Plan = Field(..., description="Plan section")
    billing_info: Optional[BillingInfo] = Field(None, description="Billing information")

    def to_html(self) -> str:
        """Render the SOAPNote as a detailed, human-readable HTML summary."""

        def highlight_null(val):
            if val is None or (isinstance(val, str) and not val.strip()):
                return "<span style='color:red;font-weight:bold'>[MISSING]</span>"
            return val

        subj = self.subjective
        obj = self.objective
        assess = self.assessment
        plan = self.plan
        billing = self.billing_info

        # Demographics
        demographics = [
            f"<b>Name:</b> {highlight_null(getattr(subj, 'patient_name', None))}",
            f"<b>Age:</b> {highlight_null(getattr(subj, 'age', None))}",
            f"<b>Gender:</b> {highlight_null(getattr(subj, 'gender', None))}",
            f"<b>Ethnicity:</b> {highlight_null(getattr(subj, 'ethnicity', None))}",
        ]
        demographics_html = "<br>".join(demographics)

        # Medical History
        med_hist = (
            "<ul>"
            + "".join(f"<li>{item}</li>" for item in subj.medical_history)
            + "</ul>"
            if subj.medical_history
            else highlight_null(None)
        )
        surg_hist = (
            "<ul>"
            + "".join(f"<li>{item}</li>" for item in (subj.surgical_history or []))
            + "</ul>"
            if subj.surgical_history
            else highlight_null(None)
        )
        fam_hist = highlight_null(getattr(subj, "family_history", None))

        # Social History
        soc = subj.social_history
        soc_hist = []
        if soc:
            soc_hist.append(f"Living situation: {highlight_null(soc.living_situation)}")
            soc_hist.append(f"Occupation: {highlight_null(soc.occupation)}")
            soc_hist.append(
                f"Employment status: {highlight_null(soc.employment_status)}"
            )
            if soc.transportation_barriers:
                soc_hist.append("Transportation barriers")
            if soc.food_insecurity:
                soc_hist.append("Food insecurity")
            if soc.financial_strain:
                soc_hist.append("Financial strain")
            soc_hist.append(
                f"Housing stability: {highlight_null(soc.housing_stability)}"
            )
        social_history = (
            "<ul>" + "".join(f"<li>{item}</li>" for item in soc_hist) + "</ul>"
            if soc_hist
            else highlight_null(None)
        )

        # Medications
        meds = (
            "<ul>"
            + "".join(
                f"<li>{m.name} ({m.dosage}, {m.frequency}, {m.route})</li>"
                for m in subj.current_medications
            )
            + "</ul>"
            if subj.current_medications
            else highlight_null(None)
        )

        allergies = (
            "<ul>"
            + "".join(f"<li>{item}</li>" for item in (subj.allergies or []))
            + "</ul>"
            if subj.allergies
            else highlight_null(None)
        )

        ros = highlight_null(getattr(subj, "review_of_systems", None))

        # Vital Signs
        vs = obj.vital_signs
        vitals = []
        if vs:
            if (
                vs.blood_pressure_systolic is not None
                and vs.blood_pressure_diastolic is not None
            ):
                vitals.append(
                    f"Blood Pressure: {vs.blood_pressure_systolic}/{vs.blood_pressure_diastolic} mmHg"
                )
            if vs.heart_rate is not None:
                vitals.append(f"Heart Rate: {vs.heart_rate} bpm")
            if vs.respiratory_rate is not None:
                vitals.append(f"Respiratory Rate: {vs.respiratory_rate}/min")
            if vs.temperature is not None:
                vitals.append(f"Temperature: {vs.temperature} °F")
            if vs.oxygen_saturation is not None:
                vitals.append(f"SpO₂: {vs.oxygen_saturation}%")
            if vs.oxygen_saturation_on_room_air is not None:
                vitals.append(f"O₂ on room air: {vs.oxygen_saturation_on_room_air}")
            if vs.weight_kg is not None:
                vitals.append(f"Weight: {vs.weight_kg} kg")
            if vs.bmi is not None:
                vitals.append(f"BMI: {vs.bmi}")
            if vs.weight_change:
                vitals.append(f"Weight Change: {vs.weight_change}")
        vital_signs = (
            "<ul>" + "".join(f"<li>{item}</li>" for item in vitals) + "</ul>"
            if vitals
            else highlight_null(None)
        )

        # Physical Exam
        pe = obj.physical_exam
        pe_lines = []
        if pe:
            if pe.general_appearance:
                pe_lines.append(f"General: {pe.general_appearance}")
            if pe.cardiovascular:
                pe_lines.append(f"Cardiovascular: {pe.cardiovascular}")
            if pe.respiratory:
                pe_lines.append(f"Respiratory: {pe.respiratory}")
            if pe.extremities:
                pe_lines.append(f"Extremities: {pe.extremities}")
            if pe.neurological:
                pe_lines.append(f"Neurological: {pe.neurological}")
            if pe.skin:
                pe_lines.append(f"Skin: {pe.skin}")
            if pe.musculoskeletal:
                pe_lines.append(f"Musculoskeletal: {pe.musculoskeletal}")
            if pe.other_findings:
                pe_lines.append(f"Other: {pe.other_findings}")
        physical_exam = (
            "<ul>" + "".join(f"<li>{item}</li>" for item in pe_lines) + "</ul>"
            if pe_lines
            else highlight_null(None)
        )

        # Diagnostic Tests
        diag_tests = (
            "<ul>"
            + "".join(
                f"<li>{k}: {v}</li>" for k, v in (obj.diagnostic_results or {}).items()
            )
            + "</ul>"
            if obj.diagnostic_results
            else highlight_null(None)
        )

        # Assessment
        primary_dx = (
            "<ul>"
            + "".join(
                f"<li>{d.condition} (ICD-10: {d.icd10_code}, Status: {d.status}, Severity: {d.severity})</li>"
                for d in assess.primary_diagnoses
            )
            + "</ul>"
            if assess.primary_diagnoses
            else highlight_null(None)
        )
        secondary_dx = (
            "<ul>"
            + "".join(
                f"<li>{d.condition} (ICD-10: {d.icd10_code}, Status: {d.status}, Severity: {d.severity})</li>"
                for d in (assess.secondary_diagnoses or [])
            )
            + "</ul>"
            if assess.secondary_diagnoses
            else highlight_null(None)
        )
        sdh = (
            "<ul>"
            + "".join(f"<li>{item}</li>" for item in (assess.social_determinants or []))
            + "</ul>"
            if assess.social_determinants
            else highlight_null(None)
        )
        psychosocial = highlight_null(getattr(assess, "clinical_impression", None))

        # Plan
        med_plan = (
            "<ul>"
            + "".join(
                f"<li>{m.action.capitalize()} {m.medication} ({m.dosage}, {m.frequency}, Indication: {m.indication})</li>"
                for m in (plan.medications or [])
            )
            + "</ul>"
            if plan.medications
            else highlight_null(None)
        )
        orders = (
            "<ul>"
            + "".join(
                f"<li>{o.type.capitalize()}: {o.description} (Indication: {o.indication}, Timing: {o.timing})</li>"
                for o in (plan.orders or [])
            )
            + "</ul>"
            if plan.orders
            else highlight_null(None)
        )
        referrals = (
            "<ul>"
            + "".join(
                f"<li>{r.specialty}: {r.reason} (Urgency: {r.urgency})</li>"
                for r in (plan.referrals or [])
            )
            + "</ul>"
            if plan.referrals
            else highlight_null(None)
        )
        care_coord = ""
        if plan.care_coordination and plan.care_coordination.care_manager_involved:
            cm = plan.care_coordination
            services = (
                ", ".join(cm.services_coordinated) if cm.services_coordinated else ""
            )
            care_coord = (
                f"Care manager involved: {highlight_null(cm.care_manager_name)}<br>"
                f"Services coordinated: {services if services else highlight_null(None)}<br>"
                f"Home health ordered: {highlight_null(cm.home_health_ordered)}<br>"
                f"Social services referral: {highlight_null(cm.social_services_referral)}"
            )
        else:
            care_coord = highlight_null(None)
        pt_ed = (
            "<ul>"
            + "".join(f"<li>{item}</li>" for item in (plan.patient_education or []))
            + "</ul>"
            if plan.patient_education
            else highlight_null(None)
        )
        followup = (
            f"{plan.follow_up.timing}, {plan.follow_up.location}, {plan.follow_up.purpose}"
            if plan.follow_up
            else highlight_null(None)
        )
        preventive = (
            "<ul>"
            + "".join(f"<li>{item}</li>" for item in (plan.preventive_care or []))
            + "</ul>"
            if plan.preventive_care
            else highlight_null(None)
        )

        # Billing
        cpt = (
            highlight_null(getattr(billing, "cpt_code", None))
            if billing
            else highlight_null(None)
        )
        complexity = (
            highlight_null(getattr(billing, "visit_complexity", None))
            if billing
            else highlight_null(None)
        )
        icd10 = (
            "<ul>"
            + "".join(
                f"<li>{code}</li>"
                for code in (
                    billing.icd10_codes if billing and billing.icd10_codes else []
                )
            )
            + "</ul>"
            if billing and billing.icd10_codes
            else highlight_null(None)
        )

        return f"""
            <h2>SOAP Note</h2>
            <b>Date of Visit:</b> {highlight_null(getattr(self, "visit_date", None))}<br>
            <details>
              <summary><b>SUBJECTIVE</b></summary>
              <div>
                {demographics_html}<br>
                <b>Chief Complaint:</b> {highlight_null(getattr(subj, "chief_complaint", None))}<br>
                <b>History of Present Illness:</b> {highlight_null(getattr(subj, "history_present_illness", None))}<br>
                <b>Medical History:</b> {med_hist}
                <b>Surgical History:</b> {surg_hist}
                <b>Family History:</b> {fam_hist}<br>
                <b>Social History:</b> {social_history}
                <b>Current Medications:</b> {meds}
                <b>Allergies:</b> {allergies}
                <b>Review of Systems:</b> {ros}<br>
              </div>
            </details>
            <details>
              <summary><b>OBJECTIVE</b></summary>
              <div>
                <b>Vital Signs:</b> {vital_signs}
                <b>Physical Examination:</b> {physical_exam}
                <b>Diagnostic Tests:</b> {diag_tests}
              </div>
            </details>
            <details>
              <summary><b>ASSESSMENT</b></summary>
              <div>
                <b>Primary Diagnoses:</b> {primary_dx}
                <b>Secondary Diagnoses:</b> {secondary_dx}
                <b>Social Determinants of Health:</b> {sdh}
                <b>Psychosocial Factors:</b> {psychosocial}<br>
              </div>
            </details>
            <details>
              <summary><b>PLAN</b></summary>
              <div>
                <b>Medications:</b> {med_plan}
                <b>Diagnostic Orders:</b> {orders}
                <b>Referrals:</b> {referrals}
                <b>Care Coordination:</b> {care_coord}<br>
                <b>Patient Education:</b> {pt_ed}
                <b>Follow-up:</b> {followup}<br>
                <b>Preventive Care:</b> {preventive}
              </div>
            </details>
            <details>
              <summary><b>BILLING INFORMATION</b></summary>
              <div>
                <b>CPT Code:</b> {cpt}<br>
                <b>Visit Complexity:</b> {complexity}<br>
                <b>ICD-10 Codes:</b> {icd10}
              </div>
            </details>
            """

    class Config:
        json_encoders = {date: lambda v: v.isoformat()}
        json_schema_extra = {
            "example": {
                "visit_date": "2024-06-14",
                "subjective": {
                    "patient_name": "José Garcia",
                    "age": 67,
                    "gender": "male",
                    "ethnicity": "Hispanic",
                    "chief_complaint": "Burning and tingling pain in right foot",
                    "history_present_illness": "67-year-old male reports burning and tingling in right foot, especially when standing after prolonged sitting. Symptoms have worsened recently.",
                    "medical_history": [
                        "Type 2 diabetes (15 years)",
                        "Hypertension",
                        "Hyperlipidemia",
                        "CKD Stage 3",
                    ],
                    "social_history": {
                        "living_situation": "Lives alone",
                        "occupation": "Retired construction worker",
                        "transportation_barriers": True,
                        "food_insecurity": True,
                    },
                },
                "assessment": {
                    "primary_diagnoses": [
                        {
                            "condition": "Diabetic peripheral neuropathy",
                            "icd10_code": "E11.40",
                            "status": "worsening",
                        }
                    ]
                },
                "billing_info": {
                    "cpt_code": "99214",
                    "icd10_codes": ["E11.40", "I10", "N18.3", "Z59.4"],
                },
            }
        }

In [36]:
llm_with_structured_output = llm.with_structured_output(schema=SOAPNote)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_detailed),
        ("user", "{input}"),
    ]
)

response = llm_with_structured_output.invoke(prompt.format(input=user_input))

In [37]:
print(response.model_dump_json(indent=4))

{
    "visit_date": "2024-06-14",
    "subjective": {
        "patient_name": "José Garcia",
        "age": 67,
        "gender": "male",
        "ethnicity": "Hispanic",
        "chief_complaint": "Burning and tingling pain in right foot",
        "history_present_illness": "67-year-old male reports burning and tingling in right foot, especially when standing after prolonged sitting. Symptoms have worsened recently.",
        "medical_history": [
            "Type 2 diabetes (15 years)",
            "Hypertension",
            "Hyperlipidemia",
            "CKD Stage 3"
        ],
        "surgical_history": null,
        "family_history": null,
        "social_history": {
            "living_situation": "Lives alone",
            "occupation": "Retired construction worker",
            "employment_status": "Retired",
            "transportation_barriers": true,
            "food_insecurity": true,
            "financial_strain": true,
            "housing_stability": null
        },


In [38]:
print(response.to_html())


            <h2>SOAP Note</h2>
            <b>Date of Visit:</b> 2024-06-14<br>
            <details>
              <summary><b>SUBJECTIVE</b></summary>
              <div>
                <b>Name:</b> José Garcia<br><b>Age:</b> 67<br><b>Gender:</b> male<br><b>Ethnicity:</b> Hispanic<br>
                <b>Chief Complaint:</b> Burning and tingling pain in right foot<br>
                <b>History of Present Illness:</b> 67-year-old male reports burning and tingling in right foot, especially when standing after prolonged sitting. Symptoms have worsened recently.<br>
                <b>Medical History:</b> <ul><li>Type 2 diabetes (15 years)</li><li>Hypertension</li><li>Hyperlipidemia</li><li>CKD Stage 3</li></ul>
                <b>Surgical History:</b> <span style='color:red;font-weight:bold'>[MISSING]</span>
                <b>Family History:</b> <span style='color:red;font-weight:bold'>[MISSING]</span><br>
                <b>Social History:</b> <ul><li>Living situation: Lives alone</

In [35]:
display(HTML(response.to_html()))

## Shorter system prompt

In [39]:
system_prompt_short = """You are a clinical scribe. Given this doctor-patient dialogue and metadata, generate a highly accurate SOAP-formatted visit summary, capturing all minute clinical details."""

In [40]:
llm_with_structured_output = llm.with_structured_output(schema=SOAPNote)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_short),
        ("user", "{input}"),
    ]
)

response = llm_with_structured_output.invoke(prompt.format(input=user_input))

In [41]:
print(response.model_dump_json(indent=4))

{
    "visit_date": "2024-06-14",
    "subjective": {
        "patient_name": "José Garcia",
        "age": 67,
        "gender": "male",
        "ethnicity": "Hispanic",
        "chief_complaint": "Burning and tingling pain in right foot",
        "history_present_illness": "67-year-old male reports burning and tingling in right foot, especially when standing after prolonged sitting. Symptoms have worsened recently.",
        "medical_history": [
            "Type 2 diabetes (15 years)",
            "Hypertension",
            "Hyperlipidemia",
            "CKD Stage 3"
        ],
        "surgical_history": null,
        "family_history": null,
        "social_history": {
            "living_situation": "Lives alone",
            "occupation": "Retired construction worker",
            "employment_status": "Retired",
            "transportation_barriers": true,
            "food_insecurity": true,
            "financial_strain": true,
            "housing_stability": null
        },


In [42]:
display(HTML(response.to_html()))

# Testing

In [27]:
llm_with_structured_output = llm.with_structured_output(schema=SOAPNote)

In [28]:
def generate_visit_summary(dialogue):
    """
    Generate a structured SOAP visit summary from a doctor-patient dialogue.

    Args:
        dialogue (str): The doctor-patient dialogue text.

    Returns:
        SOAPNote: A structured SOAP note summarizing the visit.
    """
    response = llm_with_structured_output.invoke(prompt.format(input=dialogue))
    return response

In [29]:
dialouge = """Dr. Patel: Good afternoon, Mr. Murphy. What's bothering you most today?
Daniel Murphy: My wrists and knees are really swollen and painful in the mornings—it’s taking longer to loosen up.
Dr. Patel: Your rheumatoid arthritis might be flaring. How's your blood sugar and your stress levels?
Daniel Murphy: My recent A1C was about 7.5%. Honestly, work has been stressful, and I haven’t been exercising much.
(Nurse Scott takes vitals.)
Nurse Scott: BP 124/80, heart rate 76, weight 95 kg.
Dr. Patel: Joints indeed look swollen. Let's give you a brief steroid course and increase methotrexate slightly. I'd like physical therapy to help strengthen joints and ease pain.
Daniel Murphy: That sounds good—anything for the stress?
Dr. Patel: Yes, our care manager, Ms. Carter, will help with stress management and ergonomic tips for work.
(Care manager, Ms. Carter, briefly joins.)
Ms. Carter: Hi Daniel. We'll arrange regular check-ins and an ergonomic assessment to ease work strain.
Daniel Murphy: Great, thanks!"""

In [30]:
result = generate_visit_summary(user_input)

In [31]:
result.to_html()

"\n            <h2>SOAP Note</h2>\n            <b>Date of Visit:</b> 2024-06-14<br>\n            <details>\n              <summary><b>SUBJECTIVE</b></summary>\n              <div>\n                <b>Name:</b> José Garcia<br><b>Age:</b> 67<br><b>Gender:</b> male<br><b>Ethnicity:</b> Hispanic<br>\n                <b>Chief Complaint:</b> Burning and tingling pain in right foot<br>\n                <b>History of Present Illness:</b> 67-year-old male reports burning and tingling in right foot, especially when standing after prolonged sitting. Symptoms have worsened recently.<br>\n                <b>Medical History:</b> <ul><li>Type 2 diabetes (15 years)</li><li>Hypertension</li><li>Hyperlipidemia</li><li>CKD Stage 3</li></ul>\n                <b>Surgical History:</b> <span style='color:red;font-weight:bold'>[MISSING]</span>\n                <b>Family History:</b> <span style='color:red;font-weight:bold'>[MISSING]</span><br>\n                <b>Social History:</b> <ul><li>Living situation: 

In [32]:
import pandas as pd

df = pd.read_excel("../sample_data/input.xlsx")

In [33]:
df.head()

,Input (Transcript),Expected Output (SOAP text)
0,"Dr. Harris: Good morning, Mr. Garcia. How have...","Subjective:\n•\tPatient: José Garcia, 67-year-..."
1,"Dr. Lewis: Hi, Ms. Brown, how are things going...","Subjective:\n•\tLatisha Brown, 58-year-old Afr..."
2,"Dr. Patel: Good afternoon, Mr. Murphy. What's ...","Subjective:\n•\tDaniel Murphy, 45-year-old mal..."
3,Conversation 4: Doctor-Patient Dialogue\nDr. N...,Structured SOAP Note:\nSubjective:\n•\tMei Lin...
4,Conversation 5: Doctor-Patient Dialogue\nDr. Y...,"Subjective: Fatigue, leg swelling, history CKD..."


In [ ]:
df["Extracted JSON"] = df["Input (Transcript)"].apply(generate_visit_summary)

In [ ]:
df["Extracted JSON"] = df["Extracted JSON"].apply(lambda x: x.model_dump_json())

In [ ]:
df.to_excel("../sample_data/output.xlsx", index=False)